### Imports

In [3]:
from utils_1 import VerbParser, NounParser
import pandas as pd
import re
import os

### Loading data

In [17]:
pd.set_option('display.max_rows', None)
path = r"D:\ERC_Bang\Tasks\data_cleaning" 
os.chdir(path)
data = pd.read_csv("data.tsv", sep="\t", encoding="utf-8")

### Nouns

In [8]:
nouns = data[data["SINGULAR"].notna()]
nouns=nouns[["VALUE_ORG", "SINGULAR"]]
use=data["SINGULAR"]
use.head(10)

0       jáwdì-m
1       dá:bà-m
2      àrsɛ̌:-m
3       kɔ́mbɔ̀
4      bɛ́lɛ̀-g
5     bɛ́lɛ̀-gù
6         bɛ̀lú
7         bɛ̀lú
8    gàr sɛ̀gɛ́
9      à sɛ̀gɛ́
Name: SINGULAR, dtype: object

### Parsing nouns

In [9]:
noun_parser=NounParser()
verb_parser=VerbParser()

In [25]:
use[40:100].apply(
    lambda x: noun_parser.parse_durationals(x)
).apply(
    lambda x: noun_parser.cvcv_segmentation(x)
).apply(
    lambda x: noun_parser.nasalized_stops(x)
).apply(
lambda x: noun_parser.hyphen_space(x)
).apply(
    lambda x: verb_parser.parse_durationals(x)
)

40       ɲɔ̀ŋɔ̀-mɛ́
41      ɲ̀ɔ̀ŋɔ̀-mɛ́
42         ŋɔ̀ŋɔ́-l
43     ɔ̀gɔ̀ ɲɔ̀ŋɔ̂
44       ɲɔ́ɔ mɔ̀yⁿ
45      ɔ̀gɔ̀ ɲùwⁿó
46     ɔ̀gɔ̀ ɲɔ̀ŋɔ́
47         njè-lèbà
48        ɔ̀gɔ̀ ɲɔ̌
49           ɲòŋó-m
50       ɲɔ̀ŋɛ̀-mɛ́
51       ɲɔ̀ʔɔ̀-mɛ́
52              ɲɔ́
53     ɔ̀gɔ̀ ɲɔ̀ŋɔ̀
54      ɔ̀gɔ̀ yɔ̌ɔⁿ
55           nùmǎ-m
56     nì-nì-wⁿɛ̌-m
57            ɲáálì
58             gáàŋ
59            gâ:ŋ 
60           gám-mà
61        nì nì-wⁿé
62        nì nì-wⁿé
63          gàà-wⁿě
64             gáŋà
65             gáŋà
66        dò-ndí-yê
67             gààⁿ
68        nì nì-wⁿé
69           bàl-má
70            gàànà
71             dònó
72          gá-m-má
73           dò-ndò
74           dó-ndó
75            nìwⁿá
76             gààⁿ
77             gámà
78         nàà yî-m
79         nàà yǐ-m
80           náá bè
81      nàà-gù é-gù
82         nàŋà î-ì
83          nàŋ î:ⁿ
84          nàà wéà
85         nàà-mbîà
86        nàŋà yî-ì
87          nàà wêà
88          nàŋ î:ⁿ
89        nàà ɛ̀dɛ́


### Parsing Verbs

In [20]:
verbs = data[data["FORM"].notna()]
verbs=verbs[["VALUE_ORG", "FORM"]]
verbs=verbs["FORM"]

In [21]:
verb_parser=VerbParser()

In [22]:
verbs.head(10).apply(
    lambda x: verb_parser.parse_durationals(x)
).apply(
    lambda x: verb_parser.segment_cvcs(x)
)

424     pánd-í
425         pó
426      pál-í
427      úr-ɔ́
428    ʔɔ́r-ɛ̀
429    kɛ̀s-ɛ́
430      úr-ɛ̀
431      úr-ɛ̀
432     pɔ́-ɔ́
433      pán-á
Name: FORM, dtype: object